<img src="HotColdAnalysisDraft.jpg" alt="HotColdAnalysisDraft" style="zoom: 25%;" />

In [7]:
# original contour size 6
# we can the original contour by using canny and findContours(external)
OCNT = ['CNT0', 'CNT1', 'CNT2', 'CNT3','CNT4', 'CNT5']

# RR size 9
# we can calculate RR by radius, ex raidus = 300, img(900,900)
# drwa 9 rectangle(square), as a 3*3 square
RR = ['RR0', 'RR1', 'RR2', 'RR3', 'RR4', 'RR5', 'RR6', 'RR7', 'RR8', 'RR9']

# group Chunked contour by RR
# using RR square a sliding window, so using the RR window to get chunked contours
# by canny and findContours(external)
RRCC = [['RR0-CC0','RR0-CC1','RR0-CC2'],\
      ['RR1-CC0','RR0-CC1','RR0-CC2'],\
      ['RR2-CC0'],\
      ['RR3-CC0','RR0-CC1'],\
      ['RR4-CC0','RR0-CC1'],\
      ['RR5-CC0'],\
      ['RR6-CC0'],\
      ['RR7-CC0'],\
      ['RR8-CC0'],\
       ]

# group Chunked contour by original contour
# orginal contour should include chunked contour
# if a chunedContour could intersect with a original contour, 
# then the chunkedContour belongs to the original contour
# intersectConvexConvex  find the intersection
# so basically we don't need area for this, only need the relationship of belonging
# maybe useful when combine 2 contours in the same HotColdRegion
CNTCC = [['CNT0-RR0-CC0'],\
       ['CNT1-RR1-CC0'],\
       ['CNT2-RR1-CC1', 'CNT2-RR2-CC0'],\
       ['CNT3-RR0-CC2', 'CNT3-RR3-CC0', 'CNT3-RR6-CC0'],\
       ['CNT4-RR0-CC1', 'CNT4-RR1-CC2', 'CNT4-RR3-CC1', 'CNT4-RR4-CC0'],\
       ['CNT5-RR4-CC1', 'CNT5-RR5-CC0', 'CNT5-RR7-CC0', 'CNT5-RR8-CC0'],\
      ]

# HotColdRegion is the result based on above data
# group Chunked contour by HotColdRegion
# The biggest Chunked contour in the RR is used to mark the HoldCodeRegion
# ccArea - chunked contour area, ccArea = countourArea(chunkedContour)
# rrArea - RR square, rrArea = RR_Radius * RR_Radius
# ccrrRatio - ccArea/rrArea
#
# if the ratio is 100% and have more than 1 HC region, the top and left HC is the root HC
# if 2 contour in the same HC, combine them.

HCCC =  [['RR0-CC0-70','RR0-CC1-10','RR0-CC2-10'],\
      ['RR1-CC0-30','RR0-CC1-10','RR0-CC2-40'],\
      ['RR2-CC0-60'],\
      ['RR3-CC0-50','RR0-CC1-20'],\
      ['RR4-CC0-45','RR0-CC1-40'],\
      ['RR5-CC0-10'],\
      ['RR6-CC0-10'],\
      ['RR7-CC0-20'],\
      ['RR8-CC0-5'],\
       ]

# After compare ccrrRatio and find the biggest ccrrRatio, we get the HC
# HC could have multiple chunked contour in one, we can use it to combine contour later
# if there are 2 100 from same contour, then we use the first one
# so basically the chunked contour with original contour's relateship is necessary
HCCCResult = [['RR0-CC0-70'],\
       ['RR1-CC0-30'],\
       ['RR2-CC0-60'],\
       ['RR3-CC0-50'],\
       ['RR4-CC0-45','RR0-CC1-40'],\
      ]

In [25]:
class ChunkedContour:
    """
    ccArea - chunked contour area, ccArea = countourArea(chunkedContour)
    rrArea - RR square, rrArea = RR_Radius * RR_Radius
    ccrrRatio - ccArea/rrArea
    """
    ccrrRatio = 0
    
    'RR region index in N*N the matrix, N is the number RR region in a Row'
    rrIndex = -1
    
    'chunked contour index in the RR region'
    ccIndex = -1
    
    def __init__(self, rrIndex, ccIndex, ccrrRatio):
        self.rrIndex = rrIndex
        self.ccIndex = ccIndex
        self.ccrrRatio = ccrrRatio
        
    def __str__(self):
        return 'ChunkedContour (RR%d, CC%d, Ratio%d)' %(self.rrIndex, self.ccIndex, self.ccrrRatio)

HCMat =  [[70.12, 10, 10],\
      [30,10,40],\
      [60],\
      [50,20],\
      [45,40],\
      [10],\
      [10],\
      [20],\
      [5],\
       ]

ccMat = []
for rrIndex,rr in enumerate(HCMat):
    print(rrIndex, rr)
    for ccIndex,ccArea in enumerate(rr):
        print(ccIndex, ccArea)
        cc = ChunkedContour(rrIndex,ccIndex, ccArea)
        ccMat.append(cc)
        
for cc in ccMat:  
     print(cc)


0 [70.12, 10, 10]
0 70.12
1 10
2 10
1 [30, 10, 40]
0 30
1 10
2 40
2 [60]
0 60
3 [50, 20]
0 50
1 20
4 [45, 40]
0 45
1 40
5 [10]
0 10
6 [10]
0 10
7 [20]
0 20
8 [5]
0 5
ChunkedContour (RR0, CC0, Ratio70)
ChunkedContour (RR0, CC1, Ratio10)
ChunkedContour (RR0, CC2, Ratio10)
ChunkedContour (RR1, CC0, Ratio30)
ChunkedContour (RR1, CC1, Ratio10)
ChunkedContour (RR1, CC2, Ratio40)
ChunkedContour (RR2, CC0, Ratio60)
ChunkedContour (RR3, CC0, Ratio50)
ChunkedContour (RR3, CC1, Ratio20)
ChunkedContour (RR4, CC0, Ratio45)
ChunkedContour (RR4, CC1, Ratio40)
ChunkedContour (RR5, CC0, Ratio10)
ChunkedContour (RR6, CC0, Ratio10)
ChunkedContour (RR7, CC0, Ratio20)
ChunkedContour (RR8, CC0, Ratio5)
